In [ ]:
# USAGE
# python mixed_training.py -d Pictures

# import the necessary packages
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.optimizers import Nadam
from keras.optimizers import Adamax
from keras.optimizers import Adadelta
from keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
import matplotlib.pyplot as plt
import pandas as pd
import shap
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
import keras.backend as K
import keras

In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os

In [ ]:
inputPath = '/Users/artemsmirnov/Desktop/for_my_data/Pictures'

In [ ]:
cols = ["Y1", "X3_dummy", "X4", "X5", "X6", "X7", "X8", "X9", "X10", "X11", "X12", "X13", "X14_dummy", "logX15", "logX16", "X17", "X18", "X19", "logX20", "logX21", "X22", "X23_dummy", "X24_dummy", "X25", "X26", "X27", "X28", "X29"]

In [ ]:
df = pd.read_csv('/Users/artemsmirnov/Desktop/Pictures/alpha_alpha.txt', sep=" ", header=None, names=cols)

In [ ]:
df

In [ ]:
#now we have to extract our satellite imagery and make an array of the pictures. Every picture contains two parts. 
#The left one is the beginning of summer, the right side is the end of the summer.

def load_field_images(df, inputPath):
    # initialize our images array (i.e., the field images themselves)
    images = []

    # loop over the indexes of the fields
    for i in df.index.values:
         # find the two images for the field and sort the file paths,
        # ensuring the two are always in the *same order*
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        fieldPaths = sorted(list(glob.glob(basePath)))

        # initialize our list of input images along with the output image
        # after *combining* the two input images
        inputImages = []
        outputImage = np.zeros((64, 128, 3), dtype="uint8")


        # loop over the input field paths
        for fieldPath in fieldPaths:
            # load the input image, resize it to be 64 64, and then
            # update the list of input images
            # each photo consists of 64 pixels
            image = cv2.imread(fieldPath)
            image = cv2.resize(image, (64, 64))
            inputImages.append(image)


        # tile the two input images in the output image such the first
        # image goes in the right corner, the second image in the
        # left corner
        
        outputImage[0:64, 0:64] = inputImages[0]
        outputImage[0:64, 64:128] = inputImages[1]

        # add the tiled image to our set of images the network will be
        # trained on
        images.append(outputImage)

    # return our set of images
    return np.array(images)

In [ ]:
images = load_field_images(df, inputPath)
images = images / 255.0

In [ ]:
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainY = trainAttrX["Y1"]
testY = testAttrX["Y1"] 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(zca_whitening=True)
# fit parameters from data
datagen.fit(trainImagesX)

In [ ]:
def process_field_attributes(df, train, test):
    # initialize the column names of the continuous data
    continuous = ["X3_dummy", "X4", "X5", "X6", "X7", "X8", "X9", "X10", "X11", "X12", "X13", "X14_dummy", "logX15", "logX16", "X17", "X18", "X19", "logX20", "logX21", "X22", "X23_dummy", "X24_dummy", "X25", "X26", "X27", "X28", "X29"]

    # performin min-max scaling each continuous feature column to
    # the range [0, 1]
    cs = StandardScaler()
    trainContinuous = cs.fit_transform(train[continuous])
    testContinuous = cs.transform(test[continuous])

    # one-hot encode the zip code categorical data (by definition of
    # one-hot encoing, all output features are now in the range [0, 1])
    #zipBinarizer = LabelBinarizer().fit(df[["No"]])
    #trainCategorical = zipBinarizer.transform(train[["No"]])
    #testCategorical = zipBinarizer.transform(test[["No"]])

    # construct our training and testing data points by concatenating
    # the categorical features with the continuous features
    trainX = np.hstack([trainContinuous])
    testX = np.hstack([testContinuous])

    # return the concatenated training and testing data
    return (trainX, testX)

In [ ]:
(trainAttrX, testAttrX) = process_field_attributes(df, trainAttrX, testAttrX)

In [ ]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

In [ ]:
def create_cnn(width, height, depth, filters=(3, 32, 64), regress=False):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (height, width, depth)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)

    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs

        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(3)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)

    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)

    # check to see if the regression node should be added
    #if regress:
    x = Dense(1, activation="linear")(x)

    # construct the CNN
    model = Model(inputs, x)

    # return the CNN
    return model

In [ ]:
def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(70, input_dim=dim, activation="sigmoid"))
    model.add(Dense(25, activation="sigmoid"))

    # check to see if the regression node should be added
   # if regress:
    model.add(Dense(1, activation="linear"))

    # return our model
    return model

In [ ]:
# create the MLP and CNN models
cnn = create_cnn(128, 64, 3, regress=False)
mlp = create_mlp(trainAttrX.shape[1], regress=False)

In [ ]:
trainAttrX

In [ ]:
combinedInput = concatenate([mlp.output, cnn.output])
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

In [ ]:
gateFactor = Input(tensor = K.variable([0.3]))
model = Model(inputs=[mlp.input, cnn.input, gateFactor], outputs=x)
opt = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=True)
#opt = SGD(lr=0.001, momentum=0.0, nesterov=False)
#opt = RMSprop(lr=0.001, rho=0.9)
#opt = Adagrad(lr=0.01)
#opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999)
#opt = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999)
#opt = Adadelta(lr=0.001, rho=0.95)
model.compile(loss="mean_squared_error", optimizer=opt)
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
print("[INFO] training model...")

history = model.fit(
    [trainAttrX, trainImagesX], trainY,
    validation_data=([testAttrX, testImagesX], testY),
    epochs=100, batch_size=10, callbacks=callbacks_list, verbose=0)

In [ ]:
model.load_weights('weights-improvement-02-0.20.hdf5')

In [ ]:
# plot loss during training
plt.title('Loss / Mean Squared Error')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# make predictions on the testing data
print("[INFO] predicting FIELD ...")
preds = model.predict([testAttrX, testImagesX])
pd.DataFrame(preds).to_excel('predictions_combinedNNN.xlsx')

preds_trainY = model.predict([trainAttrX, trainImagesX])
pd.DataFrame(preds_trainY).to_excel('preds_trainY.xlsx')
pd.DataFrame(preds).to_excel('preds_testY.xlsx')

pd.DataFrame(trainY).to_excel('real_trainY.xlsx')
pd.DataFrame(testY).to_excel('real_testY.xlsx')

# compute the difference between the *predicted* field productivities and the
# *actual* field productivities, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. FIELD...: {}, std FIELD: {}".format(locale.currency(df["Y1"].mean(), grouping=True),locale.currency(df["Y1"].std(), grouping=True)))
print("[INFO] mean: {:.2f}, std: {:.2f}".format(mean, std))

In [ ]:
preds

In [ ]:
model.summary()

# CNN Part (Images)

In [ ]:
import random

In [ ]:
#i = history.history['val_loss'][-1]
#lernen = random.choice([0.01, 0.001, 0.0001, 0.00001])
#epokhi = int(random.uniform(25, 150))
#batsch = int(random.uniform(1, 15))
#    
#while i > 0.423642161533107:
#    gateFactor = Input(tensor = K.variable([0.3]))
#    model = Model(inputs=[mlp.input, cnn.input, gateFactor], outputs=x)
#    opt = Adam(lr=lernen, beta_1=0.9, beta_2=0.999, amsgrad=True)
#    model.compile(loss="mean_squared_error", optimizer=opt)
#    history = model.fit(
#    [trainAttrX, trainImagesX], trainY,
#        validation_data=([testAttrX, testImagesX], testY),
#        epochs=epokhi, batch_size=batsch)
#    i = history.history['val_loss'][-1]
#    lernen = random.choice([0.01, 0.001, 0.0001, 0.00001])
#    epokhi = int(random.uniform(25, 150))
#    batsch = int(random.uniform(1, 15)

#if i < 0.423642161533107:
#    preds = model.predict([testAttrX, testImagesX])
#    pd.DataFrame(preds).to_excel('predictions_combinedNACHT.xlsx')

Lime

In [ ]:
%load_ext autoreload
%autoreload 2
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image

In [ ]:
import time

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
inet_model = cnn

In [ ]:
#Select the 16th observation for the cheque of the LIME
time
# Hide color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels
explanation = explainer.explain_instance(images[45], inet_model.predict, top_labels=5, hide_color=0, num_samples=1000)

In [ ]:
from skimage.segmentation import mark_boundaries

In [ ]:
#Just field before the lIME

# I'm dividing by 2 and adding 0.5 because of how this Inception represents images
plt.imshow(images[45])
#preds = inet_model.predict(images)
#for x in decode_predictions(preds)[0]:
#    print(x)

In [ ]:
#There are several examples of division of the field into sectors.
#The number of superpixels is 5.

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp, mask))

In [ ]:
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(128, 64))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out)

In [ ]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
#Grey is for the part which don't have the positive impact on Y_i

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=2, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=1, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=0, hide_rest=True, min_weight=0.1)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=1, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Final picture

In [ ]:
#Дивеевский район, Нижегородская область
#For my thesis I would prefer this image. The green part has positive impact on Y_i, the orange one reduces the probability of a high level of the crop. Others don't the impact. 

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=3, hide_rest=False)
plot = plt.imshow(mark_boundaries(temp, mask))

In [ ]:
#Часть изображения, ответственная за весенний снимок, показывает недостаточную растительность для данного времени года,
#следоватлеьно, прогнозное значение урожайности для данного наблюдения по МО уменьшается из-за данного участка.

SHAP

In [ ]:
import shap
import numpy as np

In [ ]:
cnn.layers

In [ ]:
# select a train set of background examples to take an expectation over
background = trainImagesX[np.random.choice(trainImagesX.shape[0], 75, replace=False)]

# explain predictions of the model on all test images
e = shap.DeepExplainer((cnn.layers[4].input, cnn.layers[-1].output), background)
shap_values = e.shap_values(testImagesX[3:4])

# plot the feature attributions
shap.image_plot(shap_values, testImagesX[3:4], labels=None, show=True, width=200, aspect=0.2, hspace=0.2, labelpad=None)

In [ ]:
# plot the feature attributions
shap.image_plot(shap_values, testImagesX[3:4], labels=None, show=True, width=20, aspect=0.5, hspace=0.2, labelpad=None)

In [ ]:
#red pixels are the sectors which positively influence on Y_i
#blau pixels have the opposite effect
#Выберем 4-е по счету поле для примера.
#Для данного изображения (Дивеевский район, Нижегородская область) позитивное воздействие на урожайность
#оказывает река в северной части поля в оба сезона года. Негативное влияние на урожайность выявлено у юго-западного 
#участка с низкой урожайностью весной.

In [ ]:
#Интерпретаторы LIME и SHAP приходят к похожим вывожам, однако для SHAP объяснения более детальны, так как, 
#в отличие от lIME, здесь нет суперпикселей, искуственно укркупненных секторов изображения, которые 
#несколько упрощают фогорафию. Также SHAP, согласно теории, является единственным на данный момент методом,
#который предоставляет полное, а не локальное объяснение модели глубокого обучения.

# MLP Part (Tabular Data)

In [ ]:
import lime
import lime.lime_tabular
import pandas as pd

In [ ]:
preds = mlp.predict([testAttrX])
preds = pd.DataFrame(data=preds)
preds

In [ ]:
trainAttrX=pd.DataFrame(data=trainAttrX)
trainAttrX

In [ ]:
testAttrX=pd.DataFrame(data=testAttrX)
testAttrX

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(testAttrX.as_matrix(), 
                                                   feature_names=cols[1:], 
                                                   class_names=['Y1'], 
                                                   verbose = True,
                                                   mode='regression')

In [ ]:
qc = testAttrX.as_matrix()[2]
qc_reshape = qc.reshape(1,-1)
def predict(qc):
    global mlp
    qc = mlp.predict(qc)
    return qc.reshape(qc.shape)
exp = explainer.explain_instance(qc, predict, num_features=len(df.columns[1:]))

In [ ]:
results = pd.DataFrame([])
variables = ['X1','X2','logX3','X4','X5', 'logX6','X7','X8','X9','X10', 'X11', 'X12', 'X13','X14']
for i in range (0, 25):
    qc = testAttrX.as_matrix()[2]
    qc_reshape = qc.reshape(1,-1)
    def predict(qc):
        global mlp
        qc = mlp.predict(qc)
        return qc.reshape(qc.shape[0])
    exp = explainer.explain_instance(qc, predict, num_features=len(df.columns[1:]))
    #results.append(exp.as_list())
    values = []
    for var in variables:
        for item in exp.as_list():
            if var in item[0]:
                values.append(item[1])
    results[i] = values
results.head()
results = results.T
# results.columns = variables
# results.head()
#     exp.show_in_notebook(show_all=False)

In [ ]:
results

In [ ]:
results = pd.DataFrame([])
variables = ["X3_dummy", "X4", "X5", "X6", "X7", "X8", "X9", "X10", "X11", "X12", "X13", "X14_dummy", "logX15", "logX16", "X17", "X18", "X19", "logX20", "logX21", "X22", "X23_dummy", "X24_dummy", "X25", "X26", "X27", "X28", "X29"]
for i in range (0, 25):
    qc = testAttrX.as_matrix()[2]
    qc_reshape = qc.reshape(1,-1)
    def predict(qc):
        global mlp
        qc = mlp.predict(qc)
        return qc.reshape(qc.shape)
    exp = explainer.explain_instance(qc, predict, num_features=len(df.columns))
    #results.append(exp.as_list())
    values = []
    for var in variables:
        for item in exp.as_list():
            if var in item[0]:
                values.append(item[1])
    results[i] = values
results.head()
results = results.T
results.columns = variables
results.head()
#     exp.show_in_notebook(show_all=False)

In [ ]:
results.head()

In [ ]:
results.mean()

In [ ]:
exp.show_in_notebook(show_table=True, show_all=False)

In [ ]:
pd.DataFrame(results).to_excel('results_lime_combined.xlsx')

In [ ]:
testAttrX = testAttrX.as_matrix()

In [ ]:
import sklearn
import shap
from sklearn.model_selection import train_test_split

In [ ]:
# print the JS visualization code to the notebook
shap.initjs()

In [ ]:
import tensorflow as tf
#tf.compat.v1.Session()

In [ ]:
pip install tensorflow==1.14.0

In [ ]:
print(tf.__version__)
print(keras.__version__)
print(shap.__version__)

In [ ]:
testAttrX_ = pd.DataFrame(testAttrX, columns=["X3_dummy", "X4", "X5", "X6", "X7", "X8", "X9", "X10", "X11", "X12", "X13", "X14_dummy", "logX15", "logX16", "X17", "X18", "X19", "logX20", "logX21", "X22", "X23_dummy", "X24_dummy", "X25", "X26", "X27", "X28", "X29"])

In [ ]:
# select a set of background examples to take an expectation over 
background = testAttrX[np.random.choice(testAttrX.shape[0], 25, replace=False)] 
  
# explain predictions of the model on four images 
#e = shap.GradientExplainer(model, trainAttrX) 
 # ...or pass tensors directly 
e = shap.DeepExplainer((mlp.layers[0].input, mlp.layers[-1].output), background) 
shap_values = e.shap_values(testAttrX) 
  
 # plot the feature attributions 
#shap.image_plot(shap_values, trainAttrX) 
shap.force_plot(e.expected_value[0], shap_values[0][2,:], testAttrX_.iloc[2,:], link="identity")

In [ ]:
shap.force_plot(e.expected_value[0], shap_values[0], testAttrX_, link="identity") #ДЛя всех наблюдений

In [ ]:
sv = np.array(shap_values)

In [ ]:
shap.summary_plot(shap_values, testAttrX_)

In [ ]:
shap.summary_plot(shap_values, testAttrX_, plot_type='bar')

In [ ]:
sv=[]
for item in shap_values:
    for item_ in item:
        print(item_)
        a=[]
        for i in item_:
            print(i)
            a.append(i)
        sv.append(a)

In [ ]:
sv = pd.DataFrame(sv)
sv.columns = results.columns

In [ ]:
sv

In [ ]:
sv.mean()

In [ ]:
shap.dependence_plot("logX15", shap_values[0], testAttrX_)

In [ ]:
shap.summary_plot(shap_values[0], testAttrX_)